In [1]:
# Requirements:
# !pip install rtdl
# !pip install libzero==0.0.4

In [2]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [3]:
device = torch.device('cpu')
# Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
zero.improve_reproducibility(seed=123456)

123456

### Data

In [4]:
from sklearn.datasets import fetch_kddcup99 
data = fetch_kddcup99(subset='train')

from pprint import pprint
print(list(data.target_names))

['labels']


In [5]:
path = "data/DontGetKicked/training.csv"

target = "WarrantyCost"

In [6]:
df = pd.read_csv(path)
df["target"] = df.WarrantyCost# df.income
df = df.drop(target, axis=1)

In [7]:
strCol = []
for column in df.columns:
    print(column)
    if type(df[column][0]) is str:
        strCol.append(column)


RefId
IsBadBuy
PurchDate
Auction
VehYear
VehicleAge
Make
Model
Trim
SubModel
Color
Transmission
WheelTypeID
WheelType
VehOdo
Nationality
Size
TopThreeAmericanName
MMRAcquisitionAuctionAveragePrice
MMRAcquisitionAuctionCleanPrice
MMRAcquisitionRetailAveragePrice
MMRAcquisitonRetailCleanPrice
MMRCurrentAuctionAveragePrice
MMRCurrentAuctionCleanPrice
MMRCurrentRetailAveragePrice
MMRCurrentRetailCleanPrice
PRIMEUNIT
AUCGUART
BYRNO
VNZIP1
VNST
VehBCost
IsOnlineSale
target


In [8]:
strCol.remove("PurchDate")

In [9]:
# df = pd.DataFrame(data.data.tolist(), columns=data.feature_names)
oldNames = df.columns# data.feature_names

In [10]:
# df["target"] = data.target
# df["target"] = df.income

In [11]:
output = df.target.values
labels = set(output)
print('The different type of output labels are:',labels)
print('='*125)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {1028, 1543, 522, 2571, 1038, 1551, 2063, 2062, 4622, 533, 1046, 2070, 1557, 2582, 2074, 1053, 1054, 7198, 1571, 1572, 1061, 1062, 1064, 1065, 553, 3115, 2091, 2090, 1583, 6192, 2101, 1590, 569, 1086, 6208, 1601, 582, 1606, 1610, 588, 1613, 1103, 2639, 594, 3667, 1110, 1623, 2647, 1113, 2651, 3676, 1628, 1118, 1630, 1116, 1633, 1634, 1121, 2141, 1125, 1122, 2152, 1641, 1130, 620, 2157, 1134, 623, 1644, 1137, 4210, 630, 3191, 2172, 1153, 1666, 1155, 642, 4737, 1673, 650, 2700, 1169, 2193, 4241, 2198, 2711, 2712, 1689, 3222, 1181, 2206, 671, 2208, 1182, 1698, 1187, 1694, 6819, 1703, 2218, 2731, 683, 1197, 686, 1711, 2735, 693, 1717, 2230, 1205, 1209, 702, 1215, 1216, 3265, 1220, 1223, 2251, 1740, 3276, 723, 1749, 1750, 728, 1241, 1243, 2274, 1763, 1251, 3298, 1769, 2282, 1257, 4332, 1770, 1774, 2799, 2795, 754, 1270, 1272, 1785, 762, 1275, 764, 765, 2815, 1283, 1800, 1801, 1290, 1803, 1806, 782, 5392, 1808, 2322, 2836, 1301, 2838, 1303, 5908, 5913

In [12]:
def onhotencode(_df, _col):
    _values = set(_df[_col].values)
    for v in _values:
        _df[v] = _df[_col].apply(lambda x : float(x == v) )
    return _df

In [13]:
for c in df.columns:
    if (not c in strCol) and (c != "target"):
        df = df.drop(c, axis=1)

In [14]:
for col in df.columns:
    if col != "target":
        df = onhotencode(df, col)
        df = df.drop(col, axis=1)

In [15]:
df["target"] = df["target"].apply(lambda x: str(x))

In [16]:
df.to_csv("data/DontGetKicked/training_processed.csv")

In [17]:
df

,target,OTHER,ADESA,MANHEIM,DODGE,SATURN,JEEP,CADILLAC,HONDA,SCION,...,UT,NH,OK,NJ,NY,ID,CO,TX,MD,IL
0,1113,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1053,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1389,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,630,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1020,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,993,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72979,1038,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72980,1893,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72981,1974,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
list(df.columns)

['target',
 'OTHER',
 'ADESA',
 'MANHEIM',
 'DODGE',
 'SATURN',
 'JEEP',
 'CADILLAC',
 'HONDA',
 'SCION',
 'HYUNDAI',
 'CHEVROLET',
 'MERCURY',
 'NISSAN',
 'VOLKSWAGEN',
 'ISUZU',
 'INFINITI',
 'KIA',
 'MINI',
 'BUICK',
 'TOYOTA',
 'SUBARU',
 'LINCOLN',
 'PLYMOUTH',
 'HUMMER',
 'SUZUKI',
 'FORD',
 'CHRYSLER',
 'TOYOTA SCION',
 'PONTIAC',
 'MITSUBISHI',
 'OLDSMOBILE',
 'MAZDA',
 'GMC',
 'LEXUS',
 'ACURA',
 'VOLVO',
 'LEGACY',
 'SILHOUETTE 3.4L V6 S',
 'TAURUS 3.0L V6 EFI D',
 'SX4 2.0L I4 EFI',
 '1500HD SILVERADO PIC',
 'XG 350',
 'CAMRY V6 3.5L V6 SFI',
 'GALANT 4C 2.4L I4 MP',
 'FOCUS',
 'COROLLA 1.8L I-4 EFI',
 'COOPER 1.6L I-4 EFI',
 'RENDEZVOUS FWD 3.4L',
 'MURANO 2WD V6',
 'TAHOE 4WD 4.8L V8 SF',
 'ASCENDER 2WD 6C',
 'TUNDRA PICKUP 2WD V6',
 'TRACKER 2WD 4C',
 'GRAND AM 4C 2.4L I-4',
 'XTERRA 2WD V6 4.0L V',
 'SABLE',
 'COBALT 2.2L I4 MPI',
 'CENTURY V6 3.1L V6 S',
 'LIBERTY 4WD V6',
 'STRATUS V6 2.7L V6 S',
 'TACOMA PICKUP 2WD V6',
 'HIGHLANDER 4WD 4C',
 'TSX',
 'SEQUOIA 2WD',
 '